<a href="https://colab.research.google.com/github/eduardoplima/decisoes-lm/blob/main/decisoes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Uso de LLMs para análise de decisões do Tribunal de Contas do Estado do Rio Grande do Norte

Os Tribunais de Contas possuem atribuições constitucionais diversas, que abrangem desde o julgamento de contas públicas à análise de atos de pessoal (tais como admissão e aposentadoria de servidores público). Seu colegiado, composto normalmente por sete conselheiros, tem o poder de determinar multas ou obrigações de fazer a gestores públicos responsáveis por órgãos jurisdicionados que descumpram normas legais ou conformidades operacionais.

No caso específico do Tribunal de Contas do Estado do Rio Grande do Norte, sua estrutura funcional, recém modificada, contempla uma diretoria cuja função é a gestão do exame, instrução e encaminhamentos referentes a processos de controle externo. Trata-se da Diretoria de Instrução Processual, cuja subunidade denominada Coordenadoria de Controle de Decisões (CCD) é responsável pelo cadastro, controle e monitoramento das obrigações decorrentes das decisões do TCE/RN e das recomendações de fiscalização e auditoria.

Também é da incumbência da CCD o criação do Cadastro Geral de Acompanhamento de Decisões (CGAD), conforme Art. 431 do Regimento Interno do TCE/RN. O referido cadastro inclui:

* Cadastro Geral de Multas (CGM), com acompanhamento permanente dos pagamentos
diretos ao Tribunal;
* Cadastro Geral de Devoluções (CGD), para acompanhamento permanente das condenações para devolução de valores ao Tesouro Estadual e aos Tesouros Municipais;
* Cadastro Geral de Recomendações (CGR), para acompanhamento permanente de todas as decisões para obrigação de fazer ou não fazer.
* Cadastro Geral de Termos de Ajustamento de Gestão (CGTAG), para acompanhamento dos Termos de Ajustamento de Gestão celebrados pelo Ministério Público junto ao Tribunal.

Esse notebook emprega técnicas de Reconhecimento de Entidades Nomeadas, _Word embeddings_ e LLMs para a criação dos cadastros mencionados.


In [ ]:
!pip install langchain_openai langchain_community gdown

In [47]:
import os
import getpass
import gdown

import pandas as pd

In [40]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OPENAI_API_KEY:")

OPENAI_API_KEY:··········


# Dataset

O dataset utilizado é composto por decisões exaradas pelo TCE/RN no ano de 2024. Os processos do TCE/RN são compostos por informações. Cada informação representa eventos de um processo, tais como uma petição inicial, um relatório de auditoria, a juntada de um documento, ou, como o caso em tela, a decisão sobre o processo. O conjunto de dados disponível é uma lista de objetos JSON criada a partir da junção das tabelas processo e informação do banco de dados administrativo do TCE/RN. Cada objeto contém os seguintes campos:

* `numero_processo`
* `ano_processo`
* `setor`: o setor de origem da decisão, nesse caso, o gabinete do conselheiro responsável pela decisão;
* `codigo_tipo_processo`: o tipo de processo sobre o qual a decisão foi determinada. Um dataframe com as descrições de cada tipo é criado abaixo;
* `assunto`: descrição simples do processo;
* `resumo`: um resumo da informação de decisão cadastrada;
* `texto`: o texto completo de cada decisão, esse campo é obtido por meio da leitura de um arquivo pdf.

Foram mantidos no dataset apenas processos referentes a atuações do TCE/RN no controle externo, excluindo-se as decisões referentes a atos de pessoal (aposentadoria, admissão e pensão), assuntos administrativos e "outros". Esses assunto são tratados por outras diretorias na estrutura organizacional do TCE/RN (por exemplo, a Diretoria de Atos de Pessoal). Ademais, para esse experimento, só foram escolhidos processos recentes (2021 em diante).

In [109]:
url = "https://raw.githubusercontent.com/eduardoplima/decisoes-lm/refs/heads/main/dataset.json"
output = "dataset.json"
gdown.download(url, output)

url = "https://raw.githubusercontent.com/eduardoplima/decisoes-lm/refs/heads/main/tipos_processos.csv"
output = "tipos_processos.csv"
gdown.download(url, output)

Downloading...
From: https://raw.githubusercontent.com/eduardoplima/decisoes-lm/refs/heads/main/dataset.json
To: /content/dataset.json
1.20MB [00:00, 44.1MB/s]                  
Downloading...
From: https://raw.githubusercontent.com/eduardoplima/decisoes-lm/refs/heads/main/tipos_processos.csv
To: /content/tipos_processos.csv
15.7kB [00:00, 19.4MB/s]                   


'tipos_processos.csv'

In [110]:
tipos = pd.read_csv("tipos_processos.csv")

In [111]:
df = pd.read_json("dataset.json")

In [112]:
len(df)

336

In [113]:
df.head()

,numero_processo,ano_processo,codigo_tipo_processo,assunto,setor,resumo,data_resumo,arquivo,texto
0,200090,2021,APR,INADIMPLÊNCIA NA REMESSA DE SIAI DP NOVO ...,secpc,ACÓRDÃO PADRÃO,2024-02-05,secpc_200090_2021_0043.pdf,[Processo Nº 200090 / 2021 - TC (200090/2021-T...
1,200161,2022,CGE,INADIMPLÊNCIA NA REMESSA DE CONTAS ANUAIS DE G...,secsc,ACÓRDÃO PADRÃO,2024-01-17,secsc_200161_2022_0021.pdf,[Processo Nº 200161 / 2022 - TC (200161/2022-C...
2,907,2022,ACO,ACOMPANHAMENTO DE ACUMULAÇÃO DE CARGOS PÚBLICO...,secsc,ACÓRDÃO PADRÃO,2024-02-07,secsc_000907_2022_0041.pdf,[Processo Nº 000907 / 2022 - TC (000907/2022-T...
3,1264,2022,REP,REPRESENTAÇÃO EM FACE DE CONTRATAÇÕES TEMPORÁR...,secpc,ACÓRDÃO PADRÃO,2024-02-08,secpc_001264_2022_0051.pdf,[Processo Nº 001264 / 2022 - TC (001264/2022-T...
4,3101,2022,DEN,xxx ...,secsc,ACÓRDÃO PADRÃO,2024-01-25,secsc_003101_2022_0015.pdf,[Processo Nº 003101 / 2022 - TC (003101/2022-T...


O texto vem separado em uma lista, cujos índices representam páginas do pdf. Nós agrupamos a string a seguir.

In [93]:
df['texto'] = df['texto'].apply(lambda x: ''.join(x))

In [94]:
df.iloc[0]

,82
codigo_tipo_processo,LRF
assunto,PRESTAÇÃO DE CONTAS DE ACORDO COM A RESOLUÇÃO ...
setor,secsc
resumo,ACÓRDÃO PADRÃO
data_resumo,2024-03-13
arquivo,secsc_701359_2012_0038.pdf
texto,Processo Nº 701359 / 2012 - TC (701359/2012-PM...


In [95]:
print(df.iloc[0].texto)

Processo Nº 701359 / 2012 - TC (701359/2012-PMLDANTA)
SESSÃO ORDINÁRIA 00007ª, DE 12 DE MARÇO DE 2024 - 2ª CÂMARA.
Interessado(s):   PREF.MUN.LAGOA D´ANTA                                                                                                                                                                                                                                          
  
Assunto:  PRESTAÇÃO DE CONTAS DE ACORDO COM A RESOLUÇÃO 022/2011 REF. AO 
BIMESTRE: 01/2012
Responsável(is):  JOÃO PAULO GUEDES LOPES, PREFEITO - CPF:05559622451 José Batista 
Delgado - CPF:18200567400
Relator(a):  PAULO ROBERTO CHA VES ALVES
EMENTA: ANÁLISE DA GESTÃO FISCAL – EXERCÍCIO 2012. PREFEITURA 
MUNICIPAL DE LAGOA D´ANTA. PROCESSO DE PRESTAÇÃO DE CONTAS 
INSTAURADO NA VIGÊNCIA DA LCE Nº 464/2012 E PARALISADO POR MAIS 
DE TRÊS ANOS, PENDENTE DE JULGAMENTO OU DESPACHO . 
RECONHECIMENTO DA PRESCRIÇÃO TRIENAL INTERCORRENTE DA 
PRETENSÃO PUNITIV A. INTELIGÊNCIA DO ART. 111, PARÁGRAFO ÚNICO , 
DA L

In [103]:
df_lic = df[df['codigo_tipo_processo'] == 'CTV']

In [104]:
df_lic.iloc[0]

,457
codigo_tipo_processo,CTV
assunto,CARTA CONVITE Nº 06/2001 (02 VOL) ...
setor,secss
resumo,ACÓRDÃO PADRÃO
data_resumo,2024-05-13
arquivo,secss_002372_2002_0057.pdf
texto,Processo Nº 002372 / 2002 - TC (015265/2001-SE...


In [105]:
len(df_lic)

4

In [108]:
print(df_lic.iloc[3].texto)

Processo Nº 007731 / 2001 - TC (004720/2000-DATANORTE)
SESSÃO ORDINÁRIA 00032ª, DE 10 DE SETEMBRO DE 2024 - 2ª CÂMARA.
Interessado(s):   COMP.PROCES.DADOS DO RN                                                                                                                                                                                                                                        
  
Assunto:  CARTA CONVITE 001/2001 (02 VOLUMES)
Relator(a):  PAULO ROBERTO CHA VES ALVES
EMENTA: CONSTITUCIONAL. ADMINISTRATIVO. RESSARCIMENTO AO 
ERÁRIO E MULTA. TRANSCURSO DE MAIS DE CINCO ANOS DESDE A 
DECISÃO. AUSÊNCIA DE CITAÇÃO PARA PAGAMENTO. PRESCRIÇÃO DA 
PRETENSÃO EXECUTÓRIA DAS MULTAS. CONSTITUIÇÃO DE PROCESSO 
AUTÔNOMO PARA EXECUÇÃO DO RESSARCIMENTO.
ACÓRDÃO No. 229/2024 - TC
               Vistos, relatados e discutidos estes autos, em consonância com o Ministério Público 
de Contas, ACORDAM os Conselheiros, nos termos do voto proferido pelo Conselheiro 
Relator, julgar no sentido de:
a

# LLMs

Nessa seção configuramos os modelos de LLM que vamos utilizar nas seções seguintes

In [ ]:
from langchain.llms import OpenAI
llm = OpenAI(temperature=0.9)

# Reconhecimento de Entidades Nomeadas

O Reconhecimento de Entidades Nomeadas (Named Entity Recognition - NER) é uma técnica fundamental em Processamento de Linguagem Natural (PLN) que visa identificar e classificar automaticamente elementos chave em um texto, como nomes de pessoas, organizações, locais, datas, valores monetários, entre outros. Essa tarefa é essencial para o nosso objetivo, pois é imprescindível detectar os responsáveis citados e outras características da decisão.


